<a href="https://colab.research.google.com/github/amitpuri/Ask-picturize-it/blob/main/notebooks/makersuite_text_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# Install the client library and import necessary modules.
!pip install google-generativeai
import google.generativeai as palm
import base64
import json

In [ ]:
# Configure the client library by providing your API key.
palm.configure(api_key="YOUR API KEY")

In [ ]:
# These parameters for the model call can be set by URL parameters.
model = 'models/text-bison-001' # @param {isTemplate: true}
temperature = 0.6 # @param {isTemplate: true}
candidate_count = 1 # @param {isTemplate: true}
top_k = 40 # @param {isTemplate: true}
top_p = 0.95 # @param {isTemplate: true}
max_output_tokens = 1024 # @param {isTemplate: true}
text_b64 = 'U3VtbWFyaXplIHRoaXMgcGFyYWdyYXBoIGFuZCBkZXRhaWwgc29tZSByZWxldmFudCBjb250ZXh0LgoKVGV4dDogIkkgYW0gYnkgYmlydGggYSBHZW5ldmVzZSwgYW5kIG15IGZhbWlseSBpcyBvbmUgb2YgdGhlIG1vc3QgZGlzdGluZ3Vpc2hlZCBvZiB0aGF0IHJlcHVibGljLiBNeSBhbmNlc3RvcnMgaGFkIGJlZW4gZm9yIG1hbnkgeWVhcnMgY291bnNlbGxvcnMgYW5kIHN5bmRpY3MsIGFuZCBteSBmYXRoZXIgaGFkIGZpbGxlZCBzZXZlcmFsIHB1YmxpYyBzaXR1YXRpb25zIHdpdGggaG9ub3VyIGFuZCByZXB1dGF0aW9uLiBIZSB3YXMgcmVzcGVjdGVkIGJ5IGFsbCB3aG8ga25ldyBoaW0gZm9yIGhpcyBpbnRlZ3JpdHkgYW5kIGluZGVmYXRpZ2FibGUgYXR0ZW50aW9uIHRvIHB1YmxpYyBidXNpbmVzcy4gSGUgcGFzc2VkIGhpcyB5b3VuZ2VyIGRheXMgcGVycGV0dWFsbHkgb2NjdXBpZWQgYnkgdGhlIGFmZmFpcnMgb2YgaGlzIGNvdW50cnk7IGEgdmFyaWV0eSBvZiBjaXJjdW1zdGFuY2VzIGhhZCBwcmV2ZW50ZWQgaGlzIG1hcnJ5aW5nIGVhcmx5LCBub3Igd2FzIGl0IHVudGlsIHRoZSBkZWNsaW5lIG9mIGxpZmUgdGhhdCBoZSBiZWNhbWUgYSBodXNiYW5kIGFuZCB0aGUgZmF0aGVyIG9mIGEgZmFtaWx5LiIKClN1bW1hcnk6IEluIHRoaXMgdGV4dCwgdGhlIG5hcnJhdG9yIGlzIGRlc2NyaWJpbmcgaGlzIGJhY2tncm91bmQgYW5kIHVwYnJpbmdpbmcuIEhlIHRlbGxzIHVzIHRoYXQgaGUgaXMgYSBuYXRpdmUgb2YgR2VuZXZhLCBTd2l0emVybGFuZCwgYW5kIHRoYXQgaGlzIGZhbWlseSBpcyBvbmUgb2YgdGhlIG1vc3QgZGlzdGluZ3Vpc2hlZCBpbiB0aGUgcmVwdWJsaWMuIEhlIHRoZW4gZ29lcyBvbiB0byBkZXNjcmliZSBoaXMgZmF0aGVyLCB3aG8gd2FzIGEgcmVzcGVjdGVkIHB1YmxpYyBzZXJ2YW50LgoKVGV4dDogIlRoZSB0aGluZyB0aGUgVGltZSBUcmF2ZWxsZXIgaGVsZCBpbiBoaXMgaGFuZCB3YXMgYSBnbGl0dGVyaW5nIG1ldGFsbGljIGZyYW1ld29yaywgc2NhcmNlbHkgbGFyZ2VyIHRoYW4gYSBzbWFsbCBjbG9jaywgYW5kIHZlcnkgZGVsaWNhdGVseSBtYWRlLiBUaGVyZSB3YXMgaXZvcnkgaW4gaXQsIGFuZCBzb21lIHRyYW5zcGFyZW50IGNyeXN0YWxsaW5lIHN1YnN0YW5jZS4gQW5kIG5vdyBJIG11c3QgYmUgZXhwbGljaXQsIGZvciB0aGlzIHRoYXQgZm9sbG93c+KAlHVubGVzcyBoaXMgZXhwbGFuYXRpb24gaXMgdG8gYmUgYWNjZXB0ZWTigJRpcyBhbiBhYnNvbHV0ZWx5IHVuYWNjb3VudGFibGUgdGhpbmcuIEhlIHRvb2sgb25lIG9mIHRoZSBzbWFsbCBvY3RhZ29uYWwgdGFibGVzIHRoYXQgd2VyZSBzY2F0dGVyZWQgYWJvdXQgdGhlIHJvb20sIGFuZCBzZXQgaXQgaW4gZnJvbnQgb2YgdGhlIGZpcmUsIHdpdGggdHdvIGxlZ3Mgb24gdGhlIGhlYXJ0aHJ1Zy4iCgpTdW1tYXJ5Og==' # @param {isTemplate: true}
stop_sequences_b64 = 'W10=' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0RFUk9HQVRPUlkiLCJ0aHJlc2hvbGQiOjF9LHsiY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX1RPWElDSVRZIiwidGhyZXNob2xkIjoxfSx7ImNhdGVnb3J5IjoiSEFSTV9DQVRFR09SWV9WSU9MRU5DRSIsInRocmVzaG9sZCI6Mn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMIiwidGhyZXNob2xkIjoyfSx7ImNhdGVnb3J5IjoiSEFSTV9DQVRFR09SWV9NRURJQ0FMIiwidGhyZXNob2xkIjoyfSx7ImNhdGVnb3J5IjoiSEFSTV9DQVRFR09SWV9EQU5HRVJPVVMiLCJ0aHJlc2hvbGQiOjJ9XQ==' # @param {isTemplate: true}

# Convert the prompt text param from a bae64 string to a string.
text = base64.b64decode(text_b64).decode("utf-8")

# Convert the stop_sequences and safety_settings params from base64 strings to lists.
stop_sequences = json.loads(base64.b64decode(stop_sequences_b64).decode("utf-8"))
safety_settings = json.loads(base64.b64decode(safety_settings_b64).decode("utf-8"))

defaults = {
  'model': model,
  'temperature': temperature,
  'candidate_count': candidate_count,
  'top_k': top_k,
  'top_p': top_p,
  'max_output_tokens': max_output_tokens,
  'stop_sequences': stop_sequences,
  'safety_settings': safety_settings,
}

In [ ]:
# Call the model and print the response.
response = palm.generate_text(
  **defaults,
  prompt=text
)
print(response.result)